In [1]:
#pip install importnb

In [2]:
# This notebook executes the training routines of the agent(s) 
import random
import time
from importnb import Notebook
with Notebook():
    import Board
    import Agent
from abc import ABC, abstractmethod

In [3]:
#### Parent Class ####

In [4]:
class Game(ABC):
    def __init__(self, board, agent1):
        """
        clase abstracta Game
        """
        self.board = board
        self.agent1 = agent1

    def reset_game(self):
        self.board.reset()

    @abstractmethod
    def play(self):
        pass # To be implemented by the child classes

In [5]:
#### Child Classes ####

In [6]:
class AgentVsAgent(Game):
    def __init__(self, board, agent1, agent2, epochs = 1):
        super().__init__(board, agent1)
        self.epochs = epochs
        self.agent2 = agent2
        if agent1.chip == agent2.chip:
            raise ValueError("Los agentes deben tener fichas diferentes")

    def play(self, save_learning = True):
        """Implementa los juegos entre los dos agentes."""
        
        for epoch in range(self.epochs):
            start = time.time()
            self.reset_game()
            agent_1_turn = random.choice([True, False]) # randomize who gets first turn

            # An EPOCH
            while True :                
                
                if agent_1_turn:# Turno Agente 1
                    self.agent1.play_turn(episode = epoch)  
                        
                else: # Turno Agente 2
                    self.agent2.play_turn(episode = epoch)

                if self.board.isBoardFull() or self.board.verify_winner(self.agent1.chip) or self.board.verify_winner(self.agent2.chip):
                    break
         
                agent_1_turn = not agent_1_turn
            # END OF AN EPOCH
            
            end = time.time()
            #if epoch%epoch == 0:
            print(f"Epoch # {epoch} took {(end - start):.3f} s")
        if save_learning:
            self.agent1.export_Qtable('Q_Table_A1.pkl')
            self.agent1.export_Qtable_JSON('Q_Table_A1.json')
            self.agent2.export_Qtable('Q_Table_A2.pkl')
            self.agent2.export_Qtable_JSON('Q_Table_A2.json')

In [7]:
class PlayerVsAgent(Game):
    def __init__(self, board, agent):
        super().__init__(board, agent)
        self.agent = agent
        #self.player_chip = ?
        if self.agent.chip == "X":
            self.player_chip = "O"
        else:
            self.player_chip = "X"

    def play(self):
        """Implementa un juego entre un humano y un agente."""
                
        play_again = True
        while play_again:
            
            self.reset_game()
            player_turn = random.choice([True, False]) # randomize who gets first turn
            if player_turn:
                print("You start this time")
            else:
                print("I will start this time")
            
            while True:         
                
                if player_turn:
                    # print the current state of the board
                    self.board.print_board()
                    print("It´s your turn!")

                    # Ask for the next move and place the chip
                    while True:
                        try:
                            user_input = input("What is your move? ")
                            new_col_number = int(user_input)  # Intentar convertir a entero
                            
                            # Verificar que el número esté dentro del rango permitido
                            if not (1 <= new_col_number <= 7):
                                raise ValueError("The column must be a number between 1 and 7.")
                            
                            # Intentar colocar la ficha en el tablero
                            if not self.board.place_chip(new_col_number, self.player_chip):
                                raise ValueError("Invalid move. The column might be full or invalid. Try again.")
                            
                            # Si el movimiento es válido, romper el bucle
                            break
                            
                        except ValueError as e:
                            print(e)
                            
                    if self.board.verify_winner(self.player_chip):
                        self.board.print_board()
                        print("YOU WIN!")
                        # comunicarle al agente que perdió -> asignar la recompensa 
                        break
                    elif self.board.isBoardFull():
                        self.board.print_board()
                        print("ITS A TIE")
                        # comunicarle al agente que empato -> asignar la recompensa 
                        break

                # FIN DEL TURNO DEL Jugador
                    
                else: # Agents Turn
                    
                    print("\nIt´s My turn!")
                    self.agent.play_turn()

                    if self.board.verify_winner(self.agent.chip):
                        self.board.print_board()
                        print("YOU LOSE!")
                        break
                    elif self.board.isBoardFull():
                        self.board.print_board()
                        print("ITS A TIE")
                        # comunicarle al agente que empato -> asignar la recompensa 
                        break

                
                    # FIN DEL TURNO DEL AGENTE

                player_turn = not player_turn

            # Juego terminó, preguntar si se quiere volver a jugar
            usr_input = ""
            while usr_input.lower() not in ["y", "n", "yes", "no"]:
                usr_input = input("Do you want to play again? (y/n): ").strip()

            if usr_input.lower() in ["y", "yes"]:
                play_again = True  # No es necesario redefinir, pero preferi dejarlo explícito
            elif usr_input.lower() in ["n", "no"]:
                play_again = False

        print("Thanks for playing, Bye!")

**Entrenamiento Agente VS Agente**

In [8]:
tablero = Board.Connect4Board()

# agente = Agent.Agent(tablero, "O")
# otroAgente = Agent.Agent(tablero, "X")
# juego = AgentVsAgent(tablero, agente, otroAgente, epochs = 30)


agenteCarga = Agent.Agent(tablero, "O", filename = 'Q_Table_A1_v1')
juego = PlayerVsAgent(tablero,agenteCarga)

Q_Table_A1_v1


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



**Prueba contra un agente aleatorio**

In [9]:
juego.play()

I will start this time

It´s My turn!


        <<<<<<<<   CONNECT4  >>>>>>>>
  Col_1 Col_2 Col_3 Col_4 Col_5 Col_6 Col_7
0     -     -     -     -     -     -     -
1     -     -     -     -     -     -     -
2     -     -     -     -     -     -     -
3     -     -     -     -     -     -     -
4     -     -     -     -     -     -     -
5     -     -     -     -     O     -     -


It´s your turn!


What is your move?  1



It´s My turn!


        <<<<<<<<   CONNECT4  >>>>>>>>
  Col_1 Col_2 Col_3 Col_4 Col_5 Col_6 Col_7
0     -     -     -     -     -     -     -
1     -     -     -     -     -     -     -
2     -     -     -     -     -     -     -
3     -     -     -     -     -     -     -
4     -     -     -     -     -     -     -
5     X     -     O     -     O     -     -


It´s your turn!


What is your move?  7



It´s My turn!


        <<<<<<<<   CONNECT4  >>>>>>>>
  Col_1 Col_2 Col_3 Col_4 Col_5 Col_6 Col_7
0     -     -     -     -     -     -     -
1     -     -     -     -     -     -     -
2     -     -     -     -     -     -     -
3     -     -     -     -     -     -     -
4     -     -     -     -     -     -     O
5     X     -     O     -     O     -     X


It´s your turn!


...


It´s My turn!


        <<<<<<<<   CONNECT4  >>>>>>>>
  Col_1 Col_2 Col_3 Col_4 Col_5 Col_6 Col_7
0     -     -     -     -     O     -     X
1     -     -     -     -     X     -     O
2     O     -     -     -     O     -     X
3     X     X     X     -     O     -     O
4     O     X     O     O     O     X     X
5     X     O     O     X     X     O     X


It´s your turn!


What is your move?  6



It´s My turn!


        <<<<<<<<   CONNECT4  >>>>>>>>
  Col_1 Col_2 Col_3 Col_4 Col_5 Col_6 Col_7
0     -     -     -     -     O     -     X
1     -     -     -     -     X     -     O
2     O     -     -     -     O     -     X
3     X     X     X     O     O     X     O
4     O     X     O     O     O     X     X
5     X     O     O     X     X     O     X


YOU LOSE!


Do you want to play again? (y/n):  n


Thanks for playing, Bye!
